In [ ]:
# https://github.com/sysblok/rusvectores_tutorial/blob/master/rusvectores_tutorial.ipynb

In [7]:
# !pip install ufal.udpipe
# !pip install wget
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 250.0 kB/s eta 0:00:001m240.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 459.2 kB/s eta 0:00:001m511.1 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 572.3 kB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 314.0 kB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 419.3 kB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 447.7 kB/s eta 0:00:00 kB/s eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


Кусок кода ниже скачает рассказ О’Генри и модель UDPipe для лингвистической предобработки.
Модель весит 40 мегабайт, поэтому ячейка может выполнятся некоторое время,
особенно если у вас небыстрый интернет.

In [2]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

100% [..............................................................................] 25649 / 25649

In [ ]:
Приступим к собственно предобработке текста. Попробуем лемматизировать текст
и добавить частеречные тэги при помощи этой функции:

In [3]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

Теперь загружаем модель UDPipe, читаем текстовый файл и обрабатываем его при помощи нашей функции.
В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход мы получаем последовательность разделенных пробелами лемм с частями речи ("зеленый_NOUN трамвай_NOUN").

In [5]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

In [6]:
text = open(textfile, 'r', encoding='utf-8').read()
processed_text = tag_ud(text=text, modelfile=modelfile)
print(processed_text[:350])
with open('my_text.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


русский_PROPN  соболь_NOUN о.::генри_PROPN 
когда_SCONJ синий_ADJ как_SCONJ ночь_NOUN глаз_NOUN Молли_VERB Мак-Кивер_PROPN  класть_VERB малыш::Брэди_PROPN  на_ADP оба_NUM лопатка_NOUN он_PRON вынужденный_ADJ быть_AUX покидать_VERB ряд_NOUN банда_NOUN «Дымовый_ADJ труба»_NOUN таков_ADJ власть_NOUN нежный_ADJ укор_NOUN подружка_NOUN и_CCONJ она_PRON 


In [15]:
import sys
import logging
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

На вход модели мы даем наш обработанный текстовый файл (либо любой другой текст, важно лишь, что каждое предложение должно быть на отдельной строчке).

In [16]:
f = 'my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Параметры в скобочках:

data - данные,
size - размер вектора,
window - размер окна наблюдения,
min_count - мин. частотность слова в корпусе, которое мы берем,
sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [18]:
model = gensim.models.Word2Vec(data, vector_size=500, window=10, min_count=2, sg=0)

2024-10-11 21:24:18,704 : INFO : collecting all words and their counts
2024-10-11 21:24:18,708 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-10-11 21:24:18,712 : INFO : collected 981 word types from a corpus of 2085 raw words and 65 sentences
2024-10-11 21:24:18,713 : INFO : Creating a fresh vocabulary
2024-10-11 21:24:18,714 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 252 unique words (25.69% of original 981, drops 729)', 'datetime': '2024-10-11T21:24:18.714909', 'gensim': '4.3.3', 'python': '3.11.10 (main, Oct 10 2024, 20:25:53) [Clang 15.0.0 (clang-1500.1.0.2.5)]', 'platform': 'macOS-13.6.9-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2024-10-11 21:24:18,715 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 leaves 1356 word corpus (65.04% of original 2085, drops 729)', 'datetime': '2024-10-11T21:24:18.715878', 'gensim': '4.3.3', 'python': '3.11.10 (main, Oct 10 2024, 20:25:53) [Clang 15.0.0 (clang-1500.1.

In [19]:
model.init_sims(replace=True)

/var/folders/w7/r_mz1frd4hbdbnlm1hh0c7xxhz1_lw/T/ipykernel_18646/1992334471.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)
2024-10-11 21:25:11,866 : WARNING : destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [21]:
print(len(model.wv.index_to_key))

252


In [22]:
model.save('my.model')

2024-10-11 21:26:11,933 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'my.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-10-11T21:26:11.933725', 'gensim': '4.3.3', 'python': '3.11.10 (main, Oct 10 2024, 20:25:53) [Clang 15.0.0 (clang-1500.1.0.2.5)]', 'platform': 'macOS-13.6.9-x86_64-i386-64bit', 'event': 'saving'}
2024-10-11 21:26:11,935 : INFO : not storing attribute cum_table
2024-10-11 21:26:11,947 : INFO : saved my.model
